In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 데이터 경로 변수

In [83]:
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH

'/content/drive/MyDrive/data/'

- 시드값

In [84]:
SEED = 42

- 데이터 불러오기

In [85]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}train_tr_common_1.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}test_tr_common_1.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 12), (14940, 2), (441196, 12), (12225, 2))

- 공통 피처 파일 불러오기

In [86]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_3.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_3.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 433), (12225, 433))

# 결측치 처리

In [87]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
3회이상count,4214
구매금액표준편차,388
구매금액왜도,840
구매금액첨도,1265


In [88]:

mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
3회이상count,3284
구매금액표준편차,242
구매금액왜도,599
구매금액첨도,954


In [89]:
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)

train_ft["3회이상count"] = train_ft["3회이상count"].fillna(0)
test_ft["3회이상count"] = test_ft["3회이상count"].fillna(0)

In [90]:
train_ft["구매금액왜도"] = train_ft["구매금액왜도"].fillna(0)
train_ft["구매금액첨도"] = train_ft["구매금액첨도"].fillna(0)

test_ft["구매금액왜도"] = test_ft["구매금액왜도"].fillna(0)
test_ft["구매금액첨도"] = test_ft["구매금액첨도"].fillna(0)

In [91]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

## 구매주기+거래개월

In [92]:
train_ft['짧은_구매주기'] = train_ft['구매주기'] < train_ft['구매주기'].median()
train_ft['긴_구매주기'] = train_ft['구매주기'] > train_ft['구매주기'].median()
# train_ft['많은_거래개월수'] = train_ft['거래개월수'] > train_ft['거래개월수'].median()
# train_ft['적은_거래개월수'] = train_ft['거래개월수'] < train_ft['거래개월수'].median()

# train_ft['구매주기짧고_거래개월많음'] = train_ft['짧은_구매주기'] & train_ft['많은_거래개월수']
# train_ft['구매주기짧고_거래개월적음'] = train_ft['짧은_구매주기'] & train_ft['적은_거래개월수']
# train_ft['구매주기긴_거래개월많음'] = train_ft['긴_구매주기'] & train_ft['많은_거래개월수']
# train_ft['구매주기긴_거래개월적음'] = train_ft['긴_구매주기'] & train_ft['적은_거래개월수']

In [93]:
test_ft['짧은_구매주기'] = train_ft['구매주기'] < train_ft['구매주기'].median()
test_ft['긴_구매주기'] = train_ft['구매주기'] > train_ft['구매주기'].median()
# test_ft['많은_거래개월수'] = train_ft['거래개월수'] > train_ft['거래개월수'].median()
# test_ft['적은_거래개월수'] = train_ft['거래개월수'] < train_ft['거래개월수'].median()

# test_ft['구매주기짧고_거래개월많음'] = train_ft['짧은_구매주기'] & train_ft['많은_거래개월수']
# test_ft['구매주기짧고_거래개월적음'] = train_ft['짧은_구매주기'] & train_ft['적은_거래개월수']
# test_ft['구매주기긴_거래개월많음'] = train_ft['긴_구매주기'] & train_ft['많은_거래개월수']
# test_ft['구매주기긴_거래개월적음'] = train_ft['긴_구매주기'] & train_ft['적은_거래개월수']

## 아동 카테고리

In [94]:
train_tr.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,구매일,구매월,구매요일,구매시간,수정_중분류
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700,2004-05-01,5,5,9,차류
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000,2004-05-01,5,5,9,화장잡화
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937,2004-05-01,5,5,10,용기보증
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000,2004-05-01,5,5,10,아동복
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000,2004-05-01,5,5,10,전화기_카세트


In [95]:
train_ft.head()

,ID,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,...,시간별_거래횟수비율_17,시간별_거래횟수비율_18,시간별_거래횟수비율_19,시간별_거래횟수비율_20,시간별_거래횟수비율_21,시간별_거래횟수비율_22,최대구매액_대분류,최소구매액_대분류,짧은_구매주기,긴_구매주기
0,train_0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,0.300000,...,0.187500,0.250000,0.062500,0.00,0.0,0.0,남성정장스포츠,가정용품,False,True
1,train_1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,0.214286,...,0.121951,0.219512,0.097561,0.00,0.0,0.0,여성캐주얼,공산품,True,False
2,train_2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,0.122807,...,0.578947,0.684211,0.210526,0.00,0.0,0.0,명품잡화,명품잡화,True,False
3,train_3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,0.161137,...,1.850000,1.200000,1.100000,0.05,0.0,0.0,여성의류파트,케주얼_구두_아동,True,False
4,train_4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,0.096774,...,0.238095,0.190476,0.285714,0.00,0.0,0.0,명품잡화,공산품파트,True,False


In [96]:
tmp = train_tr[train_tr['중분류'] == '유아복'].groupby('ID')['구매가격'].count().reset_index()
train_ft = pd.merge(train_ft, tmp, on='ID', how='left').rename(columns={'구매가격': '유아복_구매횟수'})
train_ft['유아복_구매횟수'] = train_ft['유아복_구매횟수'].fillna(0)

train_ft['유아복_구매횟수_비율'] = train_ft['유아복_구매횟수'] / train_ft['거래횟수']
train_ft.head()

,ID,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,...,시간별_거래횟수비율_19,시간별_거래횟수비율_20,시간별_거래횟수비율_21,시간별_거래횟수비율_22,최대구매액_대분류,최소구매액_대분류,짧은_구매주기,긴_구매주기,유아복_구매횟수,유아복_구매횟수_비율
0,train_0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,0.300000,...,0.062500,0.00,0.0,0.0,남성정장스포츠,가정용품,False,True,0.0,0.000000
1,train_1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,0.214286,...,0.097561,0.00,0.0,0.0,여성캐주얼,공산품,True,False,0.0,0.000000
2,train_2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,0.122807,...,0.210526,0.00,0.0,0.0,명품잡화,명품잡화,True,False,0.0,0.000000
3,train_3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,0.161137,...,1.100000,0.05,0.0,0.0,여성의류파트,케주얼_구두_아동,True,False,2.0,0.010471
4,train_4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,0.096774,...,0.285714,0.00,0.0,0.0,명품잡화,공산품파트,True,False,0.0,0.000000


In [97]:
tmp = test_tr[test_tr['중분류'] == '유아복'].groupby('ID')['구매가격'].count().reset_index()
test_ft = pd.merge(test_ft, tmp, on='ID', how='left').rename(columns={'구매가격': '유아복_구매횟수'})
test_ft['유아복_구매횟수'] = test_ft['유아복_구매횟수'].fillna(0)

test_ft['유아복_구매횟수_비율'] = test_ft['유아복_구매횟수'] / test_ft['거래횟수']
test_ft.head()

,ID,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,...,시간별_거래횟수비율_19,시간별_거래횟수비율_20,시간별_거래횟수비율_21,시간별_거래횟수비율_22,최대구매액_대분류,최소구매액_대분류,짧은_구매주기,긴_구매주기,유아복_구매횟수,유아복_구매횟수_비율
0,test_0,7,5,32,0.285714,0.285714,0.428571,0.000000,1,0.142857,...,0.000000,0.0,0.0,0.0,잡화,생식품,False,True,0.0,0.0
1,test_1,4,3,54,0.000000,0.000000,0.750000,0.250000,0,0.250000,...,0.000000,0.0,0.0,0.0,남성의류,명품잡화,True,False,0.0,0.0
2,test_2,4,2,8,0.000000,0.600000,0.400000,0.000000,0,0.400000,...,0.000000,0.0,0.0,0.0,여성의류파트,잡화파트,True,False,0.0,0.0
3,test_3,28,16,16,0.062500,0.468750,0.343750,0.125000,3,0.000000,...,0.233333,0.0,0.0,0.0,패션잡화,가정용품,True,False,0.0,0.0
4,test_4,19,9,29,0.380952,0.285714,0.238095,0.095238,6,0.000000,...,0.035533,0.0,0.0,0.0,여성캐주얼,공산품,True,False,0.0,0.0


In [98]:
agg_list =[
    ('신생아_포함', lambda x: x[(x.str.contains("신생아"))].count()),
    # ('캐릭터_포함', lambda x: x[(x.str.contains("캐릭터"))].count()),
]

tmp = train_tr.groupby("ID")["중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby("ID")["중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

In [99]:
train_ft.head()

,ID,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,...,시간별_거래횟수비율_20,시간별_거래횟수비율_21,시간별_거래횟수비율_22,최대구매액_대분류,최소구매액_대분류,짧은_구매주기,긴_구매주기,유아복_구매횟수,유아복_구매횟수_비율,신생아_포함
0,train_0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,0.300000,...,0.00,0.0,0.0,남성정장스포츠,가정용품,False,True,0.0,0.000000,0
1,train_1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,0.214286,...,0.00,0.0,0.0,여성캐주얼,공산품,True,False,0.0,0.000000,0
2,train_2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,0.122807,...,0.00,0.0,0.0,명품잡화,명품잡화,True,False,0.0,0.000000,0
3,train_3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,0.161137,...,0.05,0.0,0.0,여성의류파트,케주얼_구두_아동,True,False,2.0,0.010471,0
4,train_4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,0.096774,...,0.00,0.0,0.0,명품잡화,공산품파트,True,False,0.0,0.000000,0


In [100]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [101]:
# np.mean((train_tr['대분류'] == '아동') & (train_tr['중분류'] == '유아').astype(int))

In [102]:
# agg_list = [
#             ('대분류_아동_cnt', lambda x: x.str.contains('아동').sum())
#             ]

# tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

# tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [103]:
# agg_list = [
#             # ('중분류_아동_cnt', lambda x: x.str.contains('아동').sum()),
#             ('중분류_유아_cnt', lambda x: x.str.contains('유아').sum())
#             ]

# tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')
# train_ft.head()

# tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')
# test_ft.head()

In [104]:
# train_ft['아동카테고리_구입'] = (train_ft['대분류_아동_cnt'] > 0) | (train_ft['중분류_아동_cnt'] > 0)
# test_ft['아동카테고리_구입'] = (test_ft['대분류_아동_cnt'] > 0) | (test_ft['중분류_아동_cnt'] > 0)

## 18시 이후 영캐릭터, 유아, 캐릭터, 신생아, 영캐주얼을 구매한 건수

In [105]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

In [106]:
# mask = train_tr['구매시간'] >= 18
# train_tr_mask = train_tr[mask]

# mask = test_tr['구매시간'] >= 18
# test_tr_mask = test_tr[mask]

In [107]:
# agg_list = [
#             ('유아_캐릭터_신생아_영캐주얼', lambda x: (x.isin(['영캐릭터', '유아', '캐릭터', '신생아', '영캐주얼']).mean())),
#           ]

# tmp = train_tr_mask.groupby('ID')["중분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')

# tmp = test_tr_mask.groupby('ID')["중분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')

In [108]:
# agg_list = [
#             ('유아_캐릭터_신생아_영캐주얼', lambda x: (x.isin(['유아복', '캐릭터', '신생아']).mean())),
#           ]

# tmp = train_tr_mask.groupby('ID')["중분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left')

# tmp = test_tr_mask.groupby('ID')["중분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left')

In [109]:
# train_ft['유아_캐릭터_신생아_영캐주얼'] = train_ft['유아_캐릭터_신생아_영캐주얼'].fillna(0)
# test_ft['유아_캐릭터_신생아_영캐주얼'] = test_ft['유아_캐릭터_신생아_영캐주얼'].fillna(0)

In [110]:
train_ft.shape, test_ft.shape

((14940, 438), (12225, 438))

## 내점일수+구매주기

In [111]:
train_ft['많은_내점일수'] = train_ft['내점일수'] > train_ft['내점일수'].median()
train_ft['적은_내점일수'] = train_ft['내점일수'] < train_ft['내점일수'].median()
test_ft['많은_내점일수'] = train_ft['내점일수'] > train_ft['내점일수'].median()
test_ft['적은_내점일수'] = train_ft['내점일수'] < train_ft['내점일수'].median()

train_ft['구매주기짧고_내점일수많음'] = train_ft['짧은_구매주기'] & train_ft['많은_내점일수']
train_ft['구매주기짧고_내점일수적음'] = train_ft['짧은_구매주기'] & train_ft['적은_내점일수']
train_ft['구매주기긴_내점일수많음'] = train_ft['긴_구매주기'] & train_ft['많은_내점일수']
train_ft['구매주기긴_내점일수적음'] = train_ft['긴_구매주기'] & train_ft['적은_내점일수']

test_ft['구매주기짧고_내점일수많음'] = train_ft['짧은_구매주기'] & train_ft['많은_내점일수']
test_ft['구매주기짧고_내점일수적음'] = train_ft['짧은_구매주기'] & train_ft['적은_내점일수']
test_ft['구매주기긴_내점일수많음'] = train_ft['긴_구매주기'] & train_ft['많은_내점일수']

# 0이고 차류 구매 횟수가 높을수록 미혼으로 예측할 확률이 높아진다?
test_ft['구매주기긴_내점일수적음'] = train_ft['긴_구매주기'] & train_ft['적은_내점일수']

## 주구매시간 4시 이후

In [112]:
train_ft['주구매시간_16시이후'] = train_tr['구매시간'] >= 16
test_ft['주구매시간_16시이후'] = test_tr['구매시간'] >= 16

## 방문지점수 1일 때 거래개월대비구매횟수

In [113]:
# agg_list = [
#         ('거래개월수', lambda x: x.dt.date.astype(str).str[:-3].nunique() )
#     ]

# tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [114]:
# tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

In [115]:
# train_ft["거래개월대비구매횟수"] = train_ft["거래횟수"] / train_ft["거래개월수"]
# test_ft["거래개월대비구매횟수"] = test_ft["거래횟수"] / test_ft["거래개월수"]

In [116]:
# train_ft['방문지점수1_거래대비구매5이상'] = (train_ft['방문지점수'] == 1) & (train_ft["거래개월대비구매횟수"] >= 5)
# test_ft['방문지점수1_거래대비구매5이상'] = (test_ft['방문지점수'] == 1) & (test_ft["거래개월대비구매횟수"] >= 5)

In [117]:
# train_ft = train_ft.drop(['거래개월대비구매횟수', "거래개월수"], axis=1)
# test_ft = test_ft.drop(['거래개월대비구매횟수', "거래개월수"], axis=1)

## A373000 명품잡화

In [118]:
# mask = train_tr['대분류'] == '명품잡화'
# train_tr_mask = train_tr[mask]

In [119]:
# agg_list = [
#         ('명품잡화_구매비율', lambda x: (x.isin(['명품잡화']).mean()))
#     ]

# tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [120]:
# tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

In [121]:
# train_ft['명품잡화_구매비율'] = train_ft['명품잡화_구매비율'].fillna(0)
# test_ft['명품잡화_구매비율'] = test_ft['명품잡화_구매비율'].fillna(0)

## Drop Columns

In [122]:
train_ft.drop(['요일별_거래횟수비율_0',	'요일별_거래횟수비율_1',	'요일별_거래횟수비율_2', '월화수방문비율'], axis=1)
test_ft.drop(['요일별_거래횟수비율_0',	'요일별_거래횟수비율_1',	'요일별_거래횟수비율_2', '월화수방문비율'], axis=1)

,ID,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,...,유아복_구매횟수,유아복_구매횟수_비율,신생아_포함,많은_내점일수,적은_내점일수,구매주기짧고_내점일수많음,구매주기짧고_내점일수적음,구매주기긴_내점일수많음,구매주기긴_내점일수적음,주구매시간_16시이후
0,test_0,7,5,32,0.285714,0.285714,0.428571,0.000000,1,0.142857,...,0.0,0.000000,0,False,True,False,False,False,True,False
1,test_1,4,3,54,0.000000,0.000000,0.750000,0.250000,0,0.250000,...,0.0,0.000000,0,True,False,True,False,False,False,False
2,test_2,4,2,8,0.000000,0.600000,0.400000,0.000000,0,0.400000,...,0.0,0.000000,0,True,False,True,False,False,False,False
3,test_3,28,16,16,0.062500,0.468750,0.343750,0.125000,3,0.000000,...,0.0,0.000000,0,True,False,True,False,False,False,False
4,test_4,19,9,29,0.380952,0.285714,0.238095,0.095238,6,0.000000,...,0.0,0.000000,0,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12220,test_12220,12,6,47,0.166667,0.166667,0.250000,0.416667,2,0.083333,...,0.0,0.000000,0,False,True,False,True,False,False,False
12221,test_12221,19,14,24,0.380952,0.000000,0.238095,0.380952,0,0.190476,...,6.0,0.315789,0,False,True,False,True,False,False,False
12222,test_12222,38,28,12,0.157895,0.263158,0.289474,0.289474,2,0.105263,...,0.0,0.000000,0,True,False,False,False,True,False,False
12223,test_12223,221,115,3,0.171206,0.311284,0.287938,0.229572,1,0.334630,...,30.0,0.135747,0,True,False,True,False,False,False,False


# 특성 공학(Feature Engineering)

- ID 변수 제외

In [123]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 444), (12225, 444))

- 추가 피처 만들어 보기

In [124]:
cols = [ col for col in train_ft.columns if col.startswith("수정_중_pivot_횟수비율_") ]

In [125]:
train_ft["수정_중분류별_구매횟수비율_std"] = train_ft[cols].std(axis=1)
train_ft["수정_중분류별_구매횟수비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["수정_중분류별_구매횟수비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["수정_중분류별_구매횟수비율_std"] = test_ft[cols].std(axis=1)
test_ft["수정_중분류별_구매횟수비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["수정_중분류별_구매횟수비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 447), (12225, 447))

## Feature Encoding

In [126]:
train_ft.shape, test_ft.shape

((14940, 447), (12225, 447))

In [127]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
최대구매액_대분류,28
최소구매액_대분류,28


In [128]:
%pip install category_encoders

In [129]:
import category_encoders as ce

In [130]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점","최소구매액_대분류", "최대구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점","최소구매액_대분류", "최대구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 507), (12225, 507))

In [131]:
# enc = ce.count.CountEncoder()
# train_ft["주구매_수정_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_수정_중분류"]])
# test_ft["주구매_수정_중분류_cnt"] = enc.transform(test_ft[["주구매_수정_중분류"]])

# train_ft.shape, test_ft.shape

- 문자열 피처 삭제

In [132]:
cols

['주구매지점', '최대구매액_대분류', '최소구매액_대분류']

In [133]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 504), (12225, 504))

In [134]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Inf 값 처리

In [135]:
mask = np.isinf(train_ft).sum() > 0
np.isinf(train_ft).sum()[mask]

,0


In [136]:
mask = np.isinf(test_ft).sum() > 0
np.isinf(test_ft).sum()[mask]

,0


In [137]:
cols = np.isinf(train_ft).sum()[mask].index
cols

Index([], dtype='object')

In [138]:
# train_inf = train_ft[cols].copy()
# train_inf[np.isinf(train_inf)] = np.nan  # 무한대를 NaN으로 대체
# train_inf = np.nan_to_num(train_inf, nan=np.nanmax(train_inf[~np.isinf(train_inf)]))
# np.isinf(train_inf).sum().sum()

In [139]:
# test_inf = test_ft[cols].copy()
# test_inf[np.isinf(test_inf)] = np.nan  # 무한대를 NaN으로 대체
# test_inf = np.nan_to_num(test_inf, nan=np.nanmax(test_inf[~np.isinf(test_inf)]))
# np.isinf(test_inf).sum().sum()

In [140]:
# train_ft[cols] = train_inf
# test_ft[cols] = test_inf


In [141]:
np.isinf(train_ft).sum().sum(), np.isinf(test_ft).sum().sum()

(0, 0)

## Feature Scaling

In [142]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [143]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


,거래횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,최대구매액_대분류_19,최대구매액_대분류_20,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28
0,-0.437126,-0.369867,0.002987,-1.029777,0.001191,0.838272,0.338186,0.109631,0.718557,-0.654150,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
1,0.239394,0.144110,-0.356452,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.269465,-0.542415,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
2,1.890101,1.943028,-0.869935,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.209830,0.532172,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
3,4.298510,3.793345,-1.023980,0.420933,-0.327474,0.008592,-0.135636,0.109631,-0.009001,0.054536,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
4,0.618244,0.452496,-0.613193,-0.752532,1.707410,-0.130285,-0.821561,0.693965,-0.346226,0.591326,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497


# 정답 데이터

In [144]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


In [145]:
train_ft.shape, test_ft.shape

((14940, 504), (12225, 504))

# 피처 셀렉션

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42) # 특성 선택을 하기 위한 모델
model = LogisticRegression(random_state=42) # 학습용 모델

In [ ]:
from sklearn.feature_selection import SelectFromModel
fs = SelectFromModel(rf) # 특성 선택에 사용하기 위한 모델 객체 전달해 줘야 함
x_train = fs.fit_transform(train_ft, target) # 특성 선택이 완료된 입력 데이터가 ndarray로 반환

scores = cross_val_score(model, x_train, target, cv=cv, scoring='f1_macro', n_jobs=-1)
scores.mean()

In [ ]:
best_cols = fs.get_feature_names_out()
best_cols

In [ ]:
train_tmp = train_ft[best_cols]

In [ ]:
model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model,train_tmp,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
np.mean(scores)

# cv 점수 확인해보기

In [1092]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [1093]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

In [65]:
from lightgbm import LGBMClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [1094]:
model = LGBMClassifier(random_state=SEED)
# model.fit(train_ft,target)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
np.mean(scores)

0.717874994753301

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# model = KNeighborsClassifier()
# scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
# np.mean(scores)

In [ ]:
# models = []
# models.append(model)
# models

## StratifiedKFold

In [961]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# params = {
#     "random_state": 42,
#     "n_estimators": 200,
#     "learning_rate": 0.2,
#     "n_jobs": -1
# }

# model = LGBMClassifier(**params)
# scores = cross_val_score(model,train_ft, target, cv=cv, scoring="f1", n_jobs=-1)
# scores.mean()

In [962]:
scores = []
model_list = []

for tri, vai in cv.split(train_ft,target):
    # 학습용 데이터
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    # 검증용 데이터
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    model = LGBMClassifier(max_depth=10, random_state=42) # 학습 가능한 모델 객체 생성
    model.fit(x_train, y_train) # 학습
    model_list.append(model)

    pred = model.predict_proba(x_valid)[:, 1]
    pred = (pred >= 0.5).astype(int)

    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

np.mean(scores)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.244806 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50959
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 477
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393156 -> initscore=-0.434065
[LightGBM] [Info] Start training from score -0.434065
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4699, number of negative: 7253
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117398 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50691
[LightGBM] [Info] Number of data points in the train set: 11952, number of used 

0.7174086991122335

In [ ]:
# model_list

## Voting

In [ ]:
cv

In [ ]:
estimators = [
    ("dt", DecisionTreeClassifier(random_state=SEED) ),
    ("lr", LogisticRegression(random_state=SEED) ),
    ("knn", KNeighborsClassifier() )
]

In [ ]:
parmas = {
    "estimators": estimators,
    "voting" : "soft",
    "n_jobs" : -1
}

model = VotingClassifier(**parmas)
# model.fit(train_ft, target)
# models.append(model)

scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
scores.mean()

In [ ]:
models

## Stacking

In [ ]:
cv

In [ ]:
from sklearn.ensemble import StackingClassifier
params = {
    "estimators": estimators,
    # "final_estimator": LogisticRegression(random_state=42),
    "cv" : cv,
    "n_jobs": -1
}

model = StackingClassifier(**params)
# model.fit(train_ft, target)
# models.append(model)

scores = cross_val_score(model, train_ft, target, cv=cv,scoring="f1_macro", n_jobs=-1)
scores.mean()

In [ ]:
models

## BaggingClassifier

In [ ]:
cv

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
params = {
    "random_state": 42,
    "estimator" : KNeighborsClassifier(),
    # "n_estimators" : 100,
    "n_jobs": -1
}
model = BaggingClassifier(**params)
# model.fit(train_ft, target)
scores = cross_val_score(model, train_ft,target, cv=cv, scoring="f1_macro", n_jobs=-1)
scores.mean()

In [ ]:
models.append(model)
models

# 모델 학습

In [ ]:
pred_list = []
for model in model_list:
    pred = model.predict_proba(test_ft)[:,1]
    pred_list.append(pred)

# for model in models:
#     pred = model.predict_proba(test_ft)[:,1]
#     pred_list.append(pred)

In [ ]:
pred_list

In [ ]:
pred = np.mean(pred_list, axis=0)
pred.shape

In [ ]:
pred

In [ ]:
pred = (pred >= 0.5).astype(int)

In [ ]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft,target)

In [66]:
%pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 15.9 MB/s eta 0:00:00


In [67]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.8 MB/s eta 0:00:00


In [146]:
from flaml import AutoML
auto_ml_ens = AutoML()
params = {
    "metric": "macro_f1",
    "estimator_list":['lgbm','xgboost','xgb_limitdepth','rf','extra_tree','histgb','catboost','lrl1','lrl2'],
    "task" : "classification",
    "time_budget": 60*60,
    "seed" : SEED,
    "early_stop": True, # 메타모델이 로지스틱회귀!!
    "ensemble": True
}
auto_ml_ens.fit(train_ft, target, **params)

[flaml.automl.logger: 11-11 10:07:21] {1728} INFO - task = classification
[flaml.automl.logger: 11-11 10:07:21] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 11-11 10:07:22] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-11 10:07:22] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'xgb_limitdepth', 'rf', 'extra_tree', 'histgb', 'catboost', 'lrl1', 'lrl2']
[flaml.automl.logger: 11-11 10:07:22] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 11-11 10:07:42] {2393} INFO - Estimated sufficient time budget=201038s. Estimated necessary time budget=5458s.
[flaml.automl.logger: 11-11 10:07:42] {2442} INFO -  at 26.7s,	estimator lgbm's best error=0.6223,	best estimator lgbm's best error=0.6223
[flaml.automl.logger: 11-11 10:07:42] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-11 10:07:51] {2442} INFO -  at 35.8s,	estimator lgbm's best error=0.6223,	best estimator lgbm's best error=0.622

In [147]:
pred = auto_ml_ens.predict(test_ft)

In [148]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [149]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


# 테스트 데이터 예측

In [69]:
pred = model.predict(test_ft)
pred

NameError: name 'model' is not defined

In [ ]:
pred_proba = model.predict_proba(test_ft)[:,1]
pred_proba

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [ ]:
submit

In [ ]:
submit["target"] = pred
submit

In [ ]:
submit["target"] = pred_proba
submit

- 예측 결과를 csv 파일로 저장하여 제출

In [150]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)

In [ ]:
submit.to_csv(f"{DATA_PATH}submit_proba.csv",index=False)